In [ ]:
!pip install treeple
from google.colab import files
uploaded = files.upload()
import sys
sys.path.append('/content')
import tree_metrics
from pathlib import Path
from treeple import PatchObliqueRandomForestClassifier

import numpy as np
from joblib import Parallel, delayed
from sklearn.model_selection import StratifiedShuffleSplit
from treeple import HonestForestClassifier
from treeple.datasets import (make_trunk_classification,
                              make_trunk_mixture_classification)
from treeple.stats import PermutationHonestForestClassifier, build_oob_forest
from treeple.stats.utils import _mutual_information
from treeple.tree import MultiViewDecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import os
import pandas as pd
import matplotlib.pyplot as plt
# 创建 figures 文件夹，如果它不存在
os.makedirs('/content/figures', exist_ok=True)
plt.savefig('/content/figures/might_S@98.png')

import pandas as pd
import matplotlib.pyplot as plt

import tree_metrics
from print_importance import might_importance

n_estimators = 1000
max_features = 0.3
MODEL_NAMES = {
    "might": {
        "n_estimators": n_estimators,
        "honest_fraction": 0.5,
        "n_jobs": 40,
        "bootstrap": True,
        "stratify": True,
        "max_samples": 0.6,
        "max_features": 0.3,
        "tree_estimator": MultiViewDecisionTreeClassifier(),
    },"morf": {
        "n_estimators": 100,
        "max_patch_dims": None,
        "data_dims": [2524],  # 根据您的数据维度进行调整
        "n_jobs": 40,
        "bootstrap": True,
        "max_samples": 0.6,
        "max_features": 0.3,
        "random_state": 42,
    },
}
might_kwargs = MODEL_NAMES["might"]
sample_list_file = "/content/AllSamples.MIGHT.Passed.samples.txt"
sample_list = pd.read_csv(sample_list_file, sep=" ", header=None)
sample_list.columns = ["library", "sample_id", "cohort"]
sample_list.head()
cohort2 = sample_list[sample_list["cohort"] == "Cohort2"]["sample_id"]
print(len(cohort2))
PON = sample_list[sample_list["cohort"] == "PanelOfNormals"]["sample_id"]
sample_list["cohort"].unique()
def get_X_y(f, root="", cohort=cohort2, verbose=False):
    df = pd.read_csv(root + f)
    non_features = ['Run', 'Library', 'Cancer Status', 'Tumor type', 'Stage', 'Library volume (uL)', 'Library Volume',
                    'UIDs Used', 'Experiment', 'P7', 'P7 Primer', 'MAF']
    sample_ids = df["Sample"]
    for i, sample_id in enumerate(sample_ids):
        if "." in sample_id:
            # print(sample_id.split(".")[1])
            if "Wise" in f or 'ichorCNA' in f:
                sample_ids[i] = sample_id
            else:
                sample_ids[i] = sample_id.split(".")[1]
    target = 'Cancer Status'
    y = df[target]
    y = y.replace("Healthy", 0)
    y = y.replace("Cancer", 1)
    for col in non_features:
        if col in df.columns:
            df = df.drop(col, axis=1)
    nan_cols = df.isnull().all(axis=0).to_numpy()
    # drop the columns with all nan values
    df = df.loc[:, ~nan_cols]
    if cohort is not None:
        # filter the rows with cohort1 samples
        X = df[sample_ids.isin(cohort)]
        # print(X.shape)
        y = y[sample_ids.isin(cohort)]
    else:
        X = df
    if "Wise" in f:
      X = X.fillna(0)
    X.iloc[:, 1] = X.iloc[:, 1].fillna(X.iloc[:, 1].mean(axis=0))
    nan_cols = X.isnull().all(axis=0)
    X = X.loc[:, ~nan_cols]
    if verbose:
        if nan_cols.sum() > 0:
            print(f)
            print(f"nan_cols: {nan_cols.sum()}")
            print(f"X shape: {X.shape}, y shape: {y.shape}")
        else:
            print(f)
            print(f"X shape: {X.shape}, y shape: {y.shape}")
    return X, y
def stratified_train_ml(clf, X, y):
    n_samples = X.shape[0]
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    POS = np.zeros((len(y), 3))

    for idx, (train_ix, test_ix) in enumerate(cv.split(X, y)):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]

        ### Split Training Set into Fitting Set (40%) and Calibarating Set (40%)
        train_idx = np.arange(
            X_train.shape[0]
        )  # use index array to split, so we can use the same index for the permuted array as well
        fit_idx, cal_idx = train_test_split(
            train_idx, test_size=0.5, random_state=idx, stratify=y_train
        )
        X_fit, X_cal, y_fit, y_cal = (
            X_train[fit_idx],
            X_train[cal_idx],
            y_train[fit_idx],
            y_train[cal_idx],
        )

        POS[test_ix, 0] = y_test
        clf.fit(X_fit, y_fit)
        if X_cal.shape[0] <= 1000:
            calibrated_model = CalibratedClassifierCV(
                clf, cv="prefit", method="sigmoid"
            )
        else:
            calibrated_model = CalibratedClassifierCV(
                clf, cv="prefit", method="isotonic"
            )
        calibrated_model.fit(X_cal, y_cal)
        posterior = calibrated_model.predict_proba(X_test)

        POS[test_ix, 1:] = posterior
    return clf, POS

def run_alog(f1, cohort=cohort2, model_name='might'):
    X_1, y_1 = get_X_y('{}.csv'.format(f1), cohort=cohort2, verbose=True)
    X = X_1.iloc[:, 1:]

    if model_name == 'might':
        est = HonestForestClassifier(**might_kwargs)

    elif model_name == 'morf':
        est = PatchObliqueRandomForestClassifier(**MODEL_NAMES[model_name])
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    X_combine = X.fillna(0)

    if model_name in ['might', 'morf']:
        est, posterior_arr = build_oob_forest(est, X, y_1, verbose=False)
    else:
        est, posterior_arr = stratified_train_ml(est, np.array(X_combine), np.array(y_1))

    if model_name in ['might', 'morf']:
        POS = np.nanmean(posterior_arr, axis=0)
    else:
        POS = posterior_arr


    print("POS shape:", POS.shape)
    if POS.shape[1] >= 2:
        fpr, tpr, thresholds = roc_curve(y_1, POS[:, -1], pos_label=1, drop_intermediate=False)
    else:
        print("POS 的列数不足，无法计算 roc_curve")

    S98 = np.max(tpr[fpr <= 0.02])
    tree_metrics.plot_S98(S98, fpr, tpr, model_name)

    MI = tree_metrics.Calculate_MI(model_name, y_1, POS)
    pAUC = tree_metrics.Calculate_pAUC(model_name, y_1, POS, fpr, tpr)
    hd = tree_metrics.Calculate_hd(model_name, POS)

    might_importance(model_name, est, X_combine)

    output_fname = f"{model_name}.npz"
    print(model_name, f1)
    print(model_name, S98, MI, pAUC, hd)
    np.savez_compressed(
        output_fname,
        model_name=model_name,
        y=y_1,
        S98=S98,
        posterior_arr=posterior_arr,
        MI=MI,
        pAUC=pAUC,
        hd=hd
    )
    return S98

for i in range(20):
    Parallel(n_jobs=40)(
        delayed(run_alog)(f1='WiseCondorX.Wise-1', cohort=cohort2, model_name=modelname)
        for modelname in ['might', 'morf']
    )